In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

グラフの作成に必要なライブラリ等の準備

In [ ]:
!pip install japanize-matplotlib

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import japanize_matplotlib 

In [4]:
!pip install plotly==5.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 55.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [5]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00


In [6]:
# Google Colab. やJupyter Lab.でplotlyを使用するための準備
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "colab"

from plotly.offline import iplot, plot, download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go

In [7]:
!pip install fpdf

# 「IPA」フォントをインストール
!apt-get -y install fonts-ipafont-gothic
# matplotlibのキャッシュをクリア
!rm /root/.cache/matplotlib/fontlist-v300.json

plt.rcParams['font.family']
plt.rcParams['font.family'] = 'IPAexGothic'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=993f17750e31debded060bbb250d0b51bcb7a8ab7298601324a5ade6dc720aa0
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 8,251 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-ipafont-gothic all 00303-18ubuntu1 [3,526 kB]
Get:2 http://archive.ubuntu.com/ubuntu

各データファイルの取得（Driveからファイルを取得する場合）

In [ ]:
# ブラウザでフォルダを表示するとURLのfolders以降がフォルダのIDになっている
# https://drive.google.com/drive/folders/<ID>
dir_id = "フォルダのID"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list_input = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id}).GetList()

file_list_input

In [ ]:
file_dict = {}
for f in file_list_input:
  if(f["title"] in "横浜の平均湿度.csv"):
    file_dict["横浜の平均湿度.csv"] = f["id"]
  elif(f["title"] in "横浜の平均気温.csv"):
    file_dict["横浜の平均気温.csv"] = f["id"]

In [ ]:
# 上記の方法で調べたファイルのID
file_id = file_dict["横浜の平均湿度.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 横浜の平均湿度データファイルの取得
drive_file.GetContentFile("横浜の平均湿度.csv")

average_humidity = pd.read_csv('横浜の平均湿度.csv', encoding="shift-jis", header=1)

In [ ]:
file_id = file_dict["横浜の平均気温.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 2021年の平均気温データファイルの取得
drive_file.GetContentFile("横浜の平均気温.csv")

average_temperature = pd.read_csv('横浜の平均気温.csv', encoding="shift-jis", header=1)

In [ ]:
average_temperature.dropna(inplace=True)

In [13]:
# DATEをindexに設定する
a_temp = average_temperature.set_index("DATE")
print(a_temp)

            平均気温(℃)
DATE               
2021/1/1        6.0
2021/1/2        6.5
2021/1/3        5.0
2021/1/4        6.7
2021/1/5        6.9
...             ...
2021/12/27      3.6
2021/12/28      4.5
2021/12/29      5.8
2021/12/30      7.9
2021/12/31      3.6

[365 rows x 1 columns]


In [14]:
# renameできない
a_temp.rename(columns={"DATE": "年月日"},
          inplace=True)
print(a_temp)

            平均気温(℃)
DATE               
2021/1/1        6.0
2021/1/2        6.5
2021/1/3        5.0
2021/1/4        6.7
2021/1/5        6.9
...             ...
2021/12/27      3.6
2021/12/28      4.5
2021/12/29      5.8
2021/12/30      7.9
2021/12/31      3.6

[365 rows x 1 columns]


reindexはインデックスの値を変更するのではなく振りなおす＝並べ替える関数なのでインデックスに設定したカラム名は変更できない<br />
[reindexでDataFrameの値がNaNになってしまう件](https://teratail.com/questions/240664)

In [16]:
sumple_temp =average_temperature.reindex(["年月日", "DATE", "平均気温(℃)"],
                              axis='columns')
print(sumple_temp)

     年月日        DATE  平均気温(℃)
1    NaN    2021/1/1      6.0
2    NaN    2021/1/2      6.5
3    NaN    2021/1/3      5.0
4    NaN    2021/1/4      6.7
5    NaN    2021/1/5      6.9
..   ...         ...      ...
361  NaN  2021/12/27      3.6
362  NaN  2021/12/28      4.5
363  NaN  2021/12/29      5.8
364  NaN  2021/12/30      7.9
365  NaN  2021/12/31      3.6

[365 rows x 3 columns]


In [17]:
# 一度インデックスをリセットしてからrenameする
a_temp =average_temperature.reset_index(drop=True)
a_temp.rename(columns={"DATE": "年月日"},
          inplace=True)
print(a_temp)

            年月日  平均気温(℃)
0      2021/1/1      6.0
1      2021/1/2      6.5
2      2021/1/3      5.0
3      2021/1/4      6.7
4      2021/1/5      6.9
..          ...      ...
360  2021/12/27      3.6
361  2021/12/28      4.5
362  2021/12/29      5.8
363  2021/12/30      7.9
364  2021/12/31      3.6

[365 rows x 2 columns]


In [18]:
average_humidity.dropna(inplace=True)

In [19]:
df = pd.merge(a_temp, average_humidity, on="年月日")
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364 entries, 0 to 363
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   年月日      364 non-null    object 
 1   平均気温(℃)  364 non-null    float64
 2   平均湿度(％)  364 non-null    float64
dtypes: float64(2), object(1)
memory usage: 11.4+ KB
            年月日  平均気温(℃)  平均湿度(％)
0      2021/1/1      6.0     42.0
1      2021/1/2      6.5     53.0
2      2021/1/3      5.0     51.0
3      2021/1/4      6.7     53.0
4      2021/1/5      6.9     61.0
..          ...      ...      ...
359  2021/12/27      3.6     37.0
360  2021/12/28      4.5     45.0
361  2021/12/29      5.8     53.0
362  2021/12/30      7.9     52.0
363  2021/12/31      3.6     52.0

[364 rows x 3 columns]


In [20]:
jp_font = {'fontname':'IPAGothic'} # 日本語フォント設定を作成

In [21]:
import matplotlib.font_manager as font_manager

# 2つ目の軸を表示するか否かを設定
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 第一軸のグラフ
trace1 = fig.add_trace(go.Bar(
            x=df["年月日"],
            y=df["平均気温(℃)"],
            name ="平均気温"
       ))

# 第二軸のグラフ
trace2 = fig.add_trace(go.Line(
            x=df["年月日"],
            y=df["平均湿度(％)"],
            name ="平均湿度"),
       secondary_y=True)

data = [trace1, trace2]

# グラフのタイトルと各軸のタイトル設定
fig.update_layout(
    title="<b>横浜の平均気温と平均湿度の関係</b>"
)
fig.update_yaxes(title_text="<b>平均湿度(％)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>平均気温(℃)</b>", secondary_y=True)
fig.update_xaxes(title="<b>年月日</b>")

# 文字化け防止用フォントを設定する
font = font_manager.FontProperties(
    family="IPAGothic",
    size=30
)
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


